In [2]:
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver'
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'
#driver = webdriver.Chrome(chromedriver) 

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████| 8.08M/8.08M [00:00<00:00, 38.7MB/s]
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/3219841532.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [105]:
#df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '성동구_카페 정보.csv'), sep=',')

In [5]:
df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '서울특별시 카페 정보.csv'), sep=',')

In [94]:
# df['dong'] = df.iloc[:, 3].str.split(' ').str[1]
# print(df['dong'])

In [95]:
# df.head(10)

In [6]:
df['place_name']

0         카페노티드 안국
1       토즈모임센터 종로점
2          어니언 안국점
3       블루보틀 삼청 카페
4            안녕인사동
           ...    
1120     스타벅스 상일동점
1121          까페항상
1122           빵카페
1123    룸카페쁠레쟝 천호점
1124     아이스베리 천호점
Name: place_name, Length: 1125, dtype: object

In [7]:
df['naver_keyword'] = df['place_name']
df['naver_map_url'] = ''
df.head(10)

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,1974266862,카페노티드 안국,서울 종로구 재동 86,서울 종로구 북촌로 6-3,0507-1337-9973,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1974266862,카페노티드 안국,
1,1,8574728,토즈모임센터 종로점,서울 종로구 종로2가 84-8,서울 종로구 종로 68,02-736-0061,카페,"서비스,산업 > 전문대행 > 공간대여 > 모임공간 > 토즈모임센터",http://place.map.kakao.com/8574728,토즈모임센터 종로점,
2,2,853028118,어니언 안국점,서울 종로구 계동 146-1,서울 종로구 계동길 5,070-7543-2123,카페,음식점 > 카페,http://place.map.kakao.com/853028118,어니언 안국점,
3,3,353512935,블루보틀 삼청 카페,서울 종로구 소격동 86,서울 종로구 북촌로5길 76,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/353512935,블루보틀 삼청 카페,
4,4,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울 종로구 인사동길 49,02-6954-2910,카페,음식점 > 카페 > 테마카페,http://place.map.kakao.com/1779571274,안녕인사동,
5,5,1508329235,스태픽스,서울 종로구 필운동 32-1,서울 종로구 사직로9길 22,0507-1341-2055,카페,음식점 > 카페,http://place.map.kakao.com/1508329235,스태픽스,
6,6,1429224581,아키비스트,서울 종로구 효자동 164-7,서울 종로구 효자로13길 52,02-738-1517,카페,음식점 > 카페,http://place.map.kakao.com/1429224581,아키비스트,
7,7,7912367,산모퉁이,서울 종로구 부암동 97-5,서울 종로구 백석동길 153,02-391-4737,카페,음식점 > 카페,http://place.map.kakao.com/7912367,산모퉁이,
8,8,27187492,포비 광화문점,서울 종로구 청진동 246,서울 종로구 종로3길 17,02-2251-8125,카페,음식점 > 카페,http://place.map.kakao.com/27187492,포비 광화문점,
9,9,8082652,차마시는 뜰,서울 종로구 삼청동 35-169,서울 종로구 북촌로11나길 26,02-722-7006,카페,음식점 > 카페 > 전통찻집,http://place.map.kakao.com/8082652,차마시는 뜰,


In [8]:
for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print(f'이번에 찾을 키워드 : {i}행 / 전체 {df.shape[0] -1}행중 {keyword}')
    if ' ' in keyword:
        join_keyword = '%20'.join(keyword.split())
    else:
        join_keyword = keyword
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={join_keyword}&sm=hty&style=v5"
        
        print(f'네이버 지도 url 정보: {naver_map_search_url}')

        driver.get(naver_map_search_url)
        time.sleep(2.5)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, '#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview').get_attribute('data-cid')
        
        time.sleep(1.0)
        
        #만약 검색 결과가 없다면?
    except Exception as e:
        print(e)
        pass
    
driver.quit()

이번에 찾을 키워드 : 0행 / 전체 1124행중 카페노티드 안국
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페노티드%20안국&sm=hty&style=v5
이번에 찾을 키워드 : 1행 / 전체 1124행중 토즈모임센터 종로점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=토즈모임센터%20종로점&sm=hty&style=v5
이번에 찾을 키워드 : 2행 / 전체 1124행중 어니언 안국점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=어니언%20안국점&sm=hty&style=v5
이번에 찾을 키워드 : 3행 / 전체 1124행중 블루보틀 삼청 카페
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=블루보틀%20삼청%20카페&sm=hty&style=v5
이번에 찾을 키워드 : 4행 / 전체 1124행중 안녕인사동
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=안녕인사동&sm=hty&style=v5
이번에 찾을 키워드 : 5행 / 전체 1124행중 스태픽스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=스태픽스&sm=hty&style=v5
이번에 찾을 키워드 : 6행 / 전체 1124행중 아키비스트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아키비스트&sm=hty&style=v5
이번에 찾을 키워드 : 7행 / 전체 1124행중 산모퉁이
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=산모퉁이&sm=h

이번에 찾을 키워드 : 50행 / 전체 1124행중 더하우스1932
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=더하우스1932&sm=hty&style=v5
이번에 찾을 키워드 : 51행 / 전체 1124행중 커피한약방
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=커피한약방&sm=hty&style=v5
이번에 찾을 키워드 : 52행 / 전체 1124행중 적당
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=적당&sm=hty&style=v5
이번에 찾을 키워드 : 53행 / 전체 1124행중 챔프커피 제3작업실
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=챔프커피%20제3작업실&sm=hty&style=v5
이번에 찾을 키워드 : 54행 / 전체 1124행중 리사르커피로스터스 약수점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=리사르커피로스터스%20약수점&sm=hty&style=v5
이번에 찾을 키워드 : 55행 / 전체 1124행중 섬광
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=섬광&sm=hty&style=v5
이번에 찾을 키워드 : 56행 / 전체 1124행중 르풀
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=르풀&sm=hty&style=v5
이번에 찾을 키워드 : 57행 / 전체 1124행중 커피스니퍼
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=커피스니퍼&sm=hty&style=v

이번에 찾을 키워드 : 90행 / 전체 1124행중 맥심플랜트
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=맥심플랜트&sm=hty&style=v5
이번에 찾을 키워드 : 91행 / 전체 1124행중 오츠커피
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오츠커피&sm=hty&style=v5
이번에 찾을 키워드 : 92행 / 전체 1124행중 오리올
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오리올&sm=hty&style=v5
이번에 찾을 키워드 : 93행 / 전체 1124행중 사유
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=사유&sm=hty&style=v5
이번에 찾을 키워드 : 94행 / 전체 1124행중 앤트러사이트 한남점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=앤트러사이트%20한남점&sm=hty&style=v5
이번에 찾을 키워드 : 95행 / 전체 1124행중 오리지널 팬케익하우스 이태원점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오리지널%20팬케익하우스%20이태원점&sm=hty&style=v5
이번에 찾을 키워드 : 96행 / 전체 1124행중 더로열푸드앤드링크
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=더로열푸드앤드링크&sm=hty&style=v5
이번에 찾을 키워드 : 97행 / 전체 1124행중 오아시스 한남점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=오

이번에 찾을 키워드 : 124행 / 전체 1124행중 무진장
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=무진장&sm=hty&style=v5
이번에 찾을 키워드 : 125행 / 전체 1124행중 서울루덴스
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=서울루덴스&sm=hty&style=v5
이번에 찾을 키워드 : 126행 / 전체 1124행중 뉴오리진 동부이촌점
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=뉴오리진%20동부이촌점&sm=hty&style=v5
이번에 찾을 키워드 : 127행 / 전체 1124행중 널담은공간
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=널담은공간&sm=hty&style=v5
이번에 찾을 키워드 : 128행 / 전체 1124행중 삼층로비
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=삼층로비&sm=hty&style=v5
이번에 찾을 키워드 : 129행 / 전체 1124행중 코지빌라커피
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=코지빌라커피&sm=hty&style=v5
이번에 찾을 키워드 : 130행 / 전체 1124행중 챔프커피 1작업실
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=챔프커피%201작업실&sm=hty&style=v5
이번에 찾을 키워드 : 131행 / 전체 1124행중 헬카페
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=헬카페&sm=hty&sty

이번에 찾을 키워드 : 182행 / 전체 1124행중 뚝방길홍차가게
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=뚝방길홍차가게&sm=hty&style=v5
이번에 찾을 키워드 : 183행 / 전체 1124행중 아르무아
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=아르무아&sm=hty&style=v5
이번에 찾을 키워드 : 184행 / 전체 1124행중 인덱스숍
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=인덱스숍&sm=hty&style=v5
이번에 찾을 키워드 : 185행 / 전체 1124행중 안다즈커피
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=안다즈커피&sm=hty&style=v5
이번에 찾을 키워드 : 186행 / 전체 1124행중 체리커피
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=체리커피&sm=hty&style=v5
이번에 찾을 키워드 : 187행 / 전체 1124행중 더이퀼리브리엄커피
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=더이퀼리브리엄커피&sm=hty&style=v5
이번에 찾을 키워드 : 188행 / 전체 1124행중 비에이치테이블
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=비에이치테이블&sm=hty&style=v5
이번에 찾을 키워드 : 189행 / 전체 1124행중 카페히릿
네이버 지도 url 정보: https://m.map.naver.com/search2/search.naver?query=카페히릿&sm=hty&style=v5


...

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100d39598 chromedriver + 4404632
1   chromedriver                        0x0000000100cc0fa3 chromedriver + 3911587
2   chromedriver                        0x000000010096de6a chromedriver + 425578
3   chromedriver                        0x00000001009a5c91 chromedriver + 654481
4   chromedriver                        0x00000001009a5e91 chromedriver + 654993
5   chromedriver                        0x00000001009dd274 chromedriver + 881268
6   chromedriver                        0x00000001009c3dcd chromedriver + 777677
7   chromedriver                        0x00000001009daf79 chromedriver + 872313
8   chromedriver                        0x00000001009c3bd3 chromedriver + 777171
9   chromedriver

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100d39598 chromedriver + 4404632
1   chromedriver                        0x0000000100cc0fa3 chromedriver + 3911587
2   chromedriver                        0x000000010096de6a chromedriver + 425578
3   chromedriver                        0x00000001009a5c91 chromedriver + 654481
4   chromedriver                        0x00000001009a5e91 chromedriver + 654993
5   chromedriver                        0x00000001009dd274 chromedriver + 881268
6   chromedriver                        0x00000001009c3dcd chromedriver + 777677
7   chromedriver                        0x00000001009daf79 chromedriver + 872313
8   chromedriver                        0x00000001009c3bd3 chromedriver + 777171
9   chromedriver

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100d39598 chromedriver + 4404632
1   chromedriver                        0x0000000100cc0fa3 chromedriver + 3911587
2   chromedriver                        0x000000010096de6a chromedriver + 425578
3   chromedriver                        0x00000001009a5c91 chromedriver + 654481
4   chromedriver                        0x00000001009a5e91 chromedriver + 654993
5   chromedriver                        0x00000001009dd274 chromedriver + 881268
6   chromedriver                        0x00000001009c3dcd chromedriver + 777677
7   chromedriver                        0x00000001009daf79 chromedriver + 872313
8   chromedriver                        0x00000001009c3bd3 chromedriver + 777171
9   chromedriver

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100d39598 chromedriver + 4404632
1   chromedriver                        0x0000000100cc0fa3 chromedriver + 3911587
2   chromedriver                        0x000000010096de6a chromedriver + 425578
3   chromedriver                        0x00000001009a5c91 chromedriver + 654481
4   chromedriver                        0x00000001009a5e91 chromedriver + 654993
5   chromedriver                        0x00000001009dd274 chromedriver + 881268
6   chromedriver                        0x00000001009c3dcd chromedriver + 777677
7   chromedriver                        0x00000001009daf79 chromedriver + 872313
8   chromedriver                        0x00000001009c3bd3 chromedriver + 777171
9   chromedriver

Message: no such element: Unable to locate element: {"method":"css selector","selector":"#ct > div.search_listview._content._ctList > ul > li > div.item_info > a.a_item.a_item_distance._linkSiteview"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100d39598 chromedriver + 4404632
1   chromedriver                        0x0000000100cc0fa3 chromedriver + 3911587
2   chromedriver                        0x000000010096de6a chromedriver + 425578
3   chromedriver                        0x00000001009a5c91 chromedriver + 654481
4   chromedriver                        0x00000001009a5e91 chromedriver + 654993
5   chromedriver                        0x00000001009dd274 chromedriver + 881268
6   chromedriver                        0x00000001009c3dcd chromedriver + 777677
7   chromedriver                        0x00000001009daf79 chromedriver + 872313
8   chromedriver                        0x00000001009c3bd3 chromedriver + 777171
9   chromedriver

In [9]:
df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [10]:
df['naver_map_url']

0       https://m.place.naver.com/restaurant/1229793109
1         https://m.place.naver.com/restaurant/37532328
2       https://m.place.naver.com/restaurant/1240190178
3       https://m.place.naver.com/restaurant/1942921803
4       https://m.place.naver.com/restaurant/1410492841
                             ...                       
1120              https://m.place.naver.com/restaurant/
1121              https://m.place.naver.com/restaurant/
1122              https://m.place.naver.com/restaurant/
1123              https://m.place.naver.com/restaurant/
1124              https://m.place.naver.com/restaurant/
Name: naver_map_url, Length: 1125, dtype: object

In [11]:
df = df.loc[df['naver_map_url'] != 'https://m.place.naver.com/restaurant/']
df

,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,1974266862,카페노티드 안국,서울 종로구 재동 86,서울 종로구 북촌로 6-3,0507-1337-9973,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1974266862,카페노티드 안국,https://m.place.naver.com/restaurant/1229793109
1,1,8574728,토즈모임센터 종로점,서울 종로구 종로2가 84-8,서울 종로구 종로 68,02-736-0061,카페,"서비스,산업 > 전문대행 > 공간대여 > 모임공간 > 토즈모임센터",http://place.map.kakao.com/8574728,토즈모임센터 종로점,https://m.place.naver.com/restaurant/37532328
2,2,853028118,어니언 안국점,서울 종로구 계동 146-1,서울 종로구 계동길 5,070-7543-2123,카페,음식점 > 카페,http://place.map.kakao.com/853028118,어니언 안국점,https://m.place.naver.com/restaurant/1240190178
3,3,353512935,블루보틀 삼청 카페,서울 종로구 소격동 86,서울 종로구 북촌로5길 76,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/353512935,블루보틀 삼청 카페,https://m.place.naver.com/restaurant/1942921803
4,4,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울 종로구 인사동길 49,02-6954-2910,카페,음식점 > 카페 > 테마카페,http://place.map.kakao.com/1779571274,안녕인사동,https://m.place.naver.com/restaurant/1410492841
...,...,...,...,...,...,...,...,...,...,...,...
746,26,649401592,스타벅스 구로NC,서울 구로구 구로동 573,서울 구로구 구로중앙로 152,1522-3232,카페,음식점 > 카페 > 커피전문점 > 스타벅스,http://place.map.kakao.com/649401592,스타벅스 구로NC,https://m.place.naver.com/restaurant/1650752198
747,27,27075938,스타벅스 현대디큐브2F,서울 구로구 신도림동 692,서울 구로구 경인로 662,1522-3232,카페,음식점 > 카페 > 커피전문점 > 스타벅스,http://place.map.kakao.com/27075938,스타벅스 현대디큐브2F,https://m.place.naver.com/restaurant/36693906
748,28,1566212360,그녀의커피 카페드엘르,서울 구로구 오류동 6-140,서울 구로구 고척로3길 7,02-6013-7750,카페,음식점 > 카페 > 커피전문점,http://place.map.kakao.com/1566212360,그녀의커피 카페드엘르,https://m.place.naver.com/restaurant/1332044812
749,29,1893731799,투썸플레이스 개봉역점,서울 구로구 개봉동 202-13,서울 구로구 남부순환로97길 20-1,02-2618-8133,카페,음식점 > 카페 > 커피전문점 > 투썸플레이스,http://place.map.kakao.com/1893731799,투썸플레이스 개봉역점,https://m.place.naver.com/restaurant/37912137


In [112]:
#df.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/성동구 카페 정보_url.csv')
#df.to_json('/Users/dc/practice/recommendation/recomm_drf/data/crawl/cafe/성동구 카페 정보_url.json', orient='records', indent=4, force_ascii=False)

In [12]:
df.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/서울특별시 카페 정보_url.csv')
df.to_json('/Users/dc/study_dir/recommendation/data/crawl/cafe/서울특별시 카페 정보_url.json', orient='records', indent=4, force_ascii=False)

In [121]:
# 2022.10.26
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
#df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '성동구 카페 정보_url.csv'), sep=',')
df = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe/서울특별시 카페 정보_url.csv'), sep=',')

# sample_test(2022.10.27)
#df = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/성동구 카페 정보_url_sample.csv'), sep=',')

In [14]:
df

,Unnamed: 0.1,Unnamed: 0,id,place_name,address_name,road_address_name,phone,category_group_name,category_name,place_url,naver_keyword,naver_map_url
0,0,0,1974266862,카페노티드 안국,서울 종로구 재동 86,서울 종로구 북촌로 6-3,0507-1337-9973,카페,음식점 > 카페 > 테마카페 > 디저트카페,http://place.map.kakao.com/1974266862,카페노티드 안국,https://m.place.naver.com/restaurant/1229793109
1,1,1,8574728,토즈모임센터 종로점,서울 종로구 종로2가 84-8,서울 종로구 종로 68,02-736-0061,카페,"서비스,산업 > 전문대행 > 공간대여 > 모임공간 > 토즈모임센터",http://place.map.kakao.com/8574728,토즈모임센터 종로점,https://m.place.naver.com/restaurant/37532328
2,2,2,853028118,어니언 안국점,서울 종로구 계동 146-1,서울 종로구 계동길 5,070-7543-2123,카페,음식점 > 카페,http://place.map.kakao.com/853028118,어니언 안국점,https://m.place.naver.com/restaurant/1240190178
3,3,3,353512935,블루보틀 삼청 카페,서울 종로구 소격동 86,서울 종로구 북촌로5길 76,1533-6906,카페,음식점 > 카페,http://place.map.kakao.com/353512935,블루보틀 삼청 카페,https://m.place.naver.com/restaurant/1942921803
4,4,4,1779571274,안녕인사동,서울 종로구 관훈동 155-2,서울 종로구 인사동길 49,02-6954-2910,카페,음식점 > 카페 > 테마카페,http://place.map.kakao.com/1779571274,안녕인사동,https://m.place.naver.com/restaurant/1410492841
...,...,...,...,...,...,...,...,...,...,...,...,...
721,746,26,649401592,스타벅스 구로NC,서울 구로구 구로동 573,서울 구로구 구로중앙로 152,1522-3232,카페,음식점 > 카페 > 커피전문점 > 스타벅스,http://place.map.kakao.com/649401592,스타벅스 구로NC,https://m.place.naver.com/restaurant/1650752198
722,747,27,27075938,스타벅스 현대디큐브2F,서울 구로구 신도림동 692,서울 구로구 경인로 662,1522-3232,카페,음식점 > 카페 > 커피전문점 > 스타벅스,http://place.map.kakao.com/27075938,스타벅스 현대디큐브2F,https://m.place.naver.com/restaurant/36693906
723,748,28,1566212360,그녀의커피 카페드엘르,서울 구로구 오류동 6-140,서울 구로구 고척로3길 7,02-6013-7750,카페,음식점 > 카페 > 커피전문점,http://place.map.kakao.com/1566212360,그녀의커피 카페드엘르,https://m.place.naver.com/restaurant/1332044812
724,749,29,1893731799,투썸플레이스 개봉역점,서울 구로구 개봉동 202-13,서울 구로구 남부순환로97길 20-1,02-2618-8133,카페,음식점 > 카페 > 커피전문점 > 투썸플레이스,http://place.map.kakao.com/1893731799,투썸플레이스 개봉역점,https://m.place.naver.com/restaurant/37912137


In [15]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.

# 네이버 지도의 유형 리스트
naver_map_type_list = []
# # 블로그 리뷰 리스트
# blog_review_list = []
# 블로그 리뷰 수 리스트
blog_review_qty_list = []
# 별점 점수(2022년 종료 예정) 정보 리스트
stars_list = []
# 별점 평가 수(2022년 종료 예정) 리스트
stars_qty_list = []
# 방문자 평가 참여자수 리스트
visitor_review_qty_list = []
# 방문자 리뷰 태그(호평) 리스트
visitor_review_list = ['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                       '양이 많아요', '혼밥하기 좋아요']


df2 = df[['id', 'place_name', 'address_name', 'phone']]
# df2 = pd.DataFrame(df, columns=['id', 'place_name', 'address_name', 'phone'])



for i in visitor_review_list:
    df2[i] = ''

print(df2)
# print('==============')
# df2.iloc[:, 7]




             id    place_name       address_name           phone 커피가 맛있어요  \
0    1974266862      카페노티드 안국       서울 종로구 재동 86  0507-1337-9973            
1       8574728    토즈모임센터 종로점   서울 종로구 종로2가 84-8     02-736-0061            
2     853028118       어니언 안국점    서울 종로구 계동 146-1   070-7543-2123            
3     353512935    블루보틀 삼청 카페      서울 종로구 소격동 86       1533-6906            
4    1779571274         안녕인사동   서울 종로구 관훈동 155-2    02-6954-2910            
..          ...           ...                ...             ...      ...   
721   649401592     스타벅스 구로NC     서울 구로구 구로동 573       1522-3232            
722    27075938  스타벅스 현대디큐브2F    서울 구로구 신도림동 692       1522-3232            
723  1566212360   그녀의커피 카페드엘르   서울 구로구 오류동 6-140    02-6013-7750            
724  1893731799   투썸플레이스 개봉역점  서울 구로구 개봉동 202-13    02-2618-8133            
725   963590299          잉크커피  서울 구로구 구로동 222-12             NaN            

    디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요  ... 화장실이 깨끗해요 주차하기 편해요 좌석이 편

/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[i] = ''
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/1163794770.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [16]:
dict_review = {}
for i in visitor_review_list:
    dict_review[i] = []
    
print(dict_review)

# dict_review ={'커피가 맛있어요' : [], '빵이 맛있어요' : [], '친절해요' : []}

{'커피가 맛있어요': [], '디저트가 맛있어요': [], '빵이 맛있어요': [], '친절해요': [], '인테리어가 멋져요': [], '매장이 청결해요': [], '뷰가 좋아요': [], '음료가 맛있어요': [], '사진이 잘 나와요': [], '특별한 메뉴가 있어요': [], '대화하기 좋아요': [], '가성비가 좋아요': [], '집중하기 좋아요': [], '화장실이 깨끗해요': [], '주차하기 편해요': [], '좌석이 편해요': [], '음식이 맛있어요': [], '매장이 넓어요': [], '단체모임 하기 좋아요': [], '특별한 날 가기 좋아요': [], '재료가 신선해요': [], '양이 많아요': [], '혼밥하기 좋아요': []}


In [17]:
#chromedriver = '/Users/dc/practice/recommendation/recomm_drf/cafe/chromedriver' 
#chromedriver = '/Users/dc/study_dir/recommendation/cafe/chromedriver'

# 메인 드라이버 : 별점 등을 크롤링
#driver = webdriver.Chrome(chromedriver) 
driver = webdriver.Chrome(ChromeDriverManager().install())

# 서브 드라이버 : 리뷰 텍스트를 리뷰 탭 들어가서 크롤링
#sub_driver = webdriver.Chrome(chromedriver)
sub_driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/209267742.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/209267742.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  sub_driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
# bs4 이용

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    time.sleep(3.0)
    
    html_driver = driver.page_source
    soup1 = BeautifulSoup(html_driver, 'html.parser')
    
    html_sub_driver = sub_driver.page_source
    soup2 = BeautifulSoup(html_sub_driver, 'html.parser')
    
    time.sleep(2.0)
    
    print('============='*8)
    print(f'{i+1}번째 데이터 수집 시작')
    
    # 네이버 지도의 유형 분류
    try: 
        naver_map_type = soup1.select_one(".DJJvD").text
        print(f'naver_map_type: {naver_map_type}')
        naver_map_type_list.append(naver_map_type)    
    except:
        print('Element 요소 미존재')
        naver_map_type_list.append('null')
        
    print(f'naver_map_type_list 길이: {len(naver_map_type_list)}')
        
    time.sleep(1.0)
        
    # 블로그 리뷰 수
    try:
        blog_review_qty = soup1.select(".place_bluelink>em")[1].text
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
    except:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
        
    print(f'blog_review_qty_list 길이: {len(blog_review_qty_list)}')
        
    time.sleep(1.0)
        
    # 별점 점수(2022년 종료 예정)
    try:
        star_review_stars = soup1.select_one("div.dAsGb>span.PXMot.LXIwF>em").text 
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
    except:
        print('Element 요소 미존재')
        stars_list.append('null')
        
    print(f'stars_list 길이: {len(stars_list)}')
        
    time.sleep(0.5)
        
    # 별점 평가 수(2022년 종료 예정)
    try:
        star_review_qty = soup2.select("div.Xj_yJ>span.m7jAR")[1]
        star_review_qty = star_review_qty.text
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
    except:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
        
    print(f'stars_qty_list 길이: {len(stars_qty_list)}')
        
    time.sleep(0.5)
        
    # 방문자 평가 참여자수 + # 방문자 호평 부분
    try: # 방문자 평가 참여자수
        visitor_review_qty = soup2.select_one("div._Wmab>span.BPjKc").text
        visitor_review_qty = re.sub(r'[()]', '', visitor_review_qty)
        visitor_review_qty = re.sub(r'명 참여', '', visitor_review_qty)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        
        # 방문자 호평 부분
        exist_review = []

        visitor_eval_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.nWiXa")
        visitor_choice_list = soup2.select("div.k2tmh>ul.uNsI9>li.nbD78>div.CsBE9>span.TwM9q")

        for i in range(5):
            visitor_eval = visitor_eval_list[i].text
            text_evaluation = re.sub('["]', '', visitor_eval)
            #print(f'방문자 호평 부분: {text_evaluation}')
            visitor_choice = visitor_choice_list[i].text
            visitor_choice = re.sub('이 키워드를 선택한 인원', '', visitor_choice)
            visitor_choice = visitor_choice.strip()
            #print(f'방문자 호평수: {visitor_choice}')
            if text_evaluation in visitor_review_list:
                dict_review[text_evaluation].append(visitor_choice)
                exist_review.append(text_evaluation)
                print(f'해당 플레이스 리뷰: {exist_review}')
        for review in visitor_review_list:
            if review not in exist_review:
                dict_review[review].append('')
        
    except:
        # 방문자 평가 참여자수
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)
        # 방문자 호평 부분
        for i in visitor_review_list:
            dict_review[i].append('')

/var/folders/sv/q7djt_49263gm3db04b4r4rr0000gn/T/ipykernel_76570/2628063239.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):


  0%|          | 0/726 [00:00<?, ?it/s]

1번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 1
블로그 리뷰 수: 3,010
blog_review_qty_list 길이: 1
별점 점수(2022년 종료 예정): 4.44
stars_list 길이: 1
별점 평가 수(2022년 종료 예정): 4,894개 평점 6,027명 참여
stars_qty_list 길이: 1
방문자 평가 참여자수: 3,929
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요', '매장이 청결해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요', '매장이 청결해요', '인테리어가 멋져요']
2번째 데이터 수집 시작
naver_map_type: 장소대여
naver_map_type_list 길이: 2
블로그 리뷰 수: 33
blog_review_qty_list 길이: 2
별점 점수(2022년 종료 예정): 4.33
stars_list 길이: 2
별점 평가 수(2022년 종료 예정): 108개 평점 102명 참여
stars_qty_list 길이: 2
방문자 평가 참여자수 10명 미만인 장소
3번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 3
블로그 리뷰 수: 3,553
blog_review_qty_list 길이: 3
별점 점수(2022년 종료 예정): 4.43
stars_list 길이: 3
별점 평가 수(2022년 종료 예정): 4,053개 평점 5,691명 참여
stars_qty_list 길이: 3
방문자 평가 참여자수: 3,671
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '인테리어가 멋져요']
해당 

블로그 리뷰 수: 599
blog_review_qty_list 길이: 16
별점 점수(2022년 종료 예정): 4.45
stars_list 길이: 16
별점 평가 수(2022년 종료 예정): 413개 평점 474명 참여
stars_qty_list 길이: 16
방문자 평가 참여자수: 20
해당 플레이스 리뷰: ['사진이 잘 나와요']
해당 플레이스 리뷰: ['사진이 잘 나와요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['사진이 잘 나와요', '인테리어가 멋져요', '친절해요']
해당 플레이스 리뷰: ['사진이 잘 나와요', '인테리어가 멋져요', '친절해요', '커피가 맛있어요']
해당 플레이스 리뷰: ['사진이 잘 나와요', '인테리어가 멋져요', '친절해요', '커피가 맛있어요', '좌석이 편해요']
17번째 데이터 수집 시작
naver_map_type: 브런치
naver_map_type_list 길이: 17
블로그 리뷰 수: 779
blog_review_qty_list 길이: 17
별점 점수(2022년 종료 예정): 4.36
stars_list 길이: 17
별점 평가 수(2022년 종료 예정): 956개 평점 1,245명 참여
stars_qty_list 길이: 17
방문자 평가 참여자수: 758
해당 플레이스 리뷰: ['음식이 맛있어요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '매장이 넓어요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '매장이 넓어요', '단체모임 하기 좋아요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '매장이 넓어요', '단체모임 하기 좋아요', '친절해요']
18번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 18
블로그 리뷰 수: 2,151
blog_review_qty_list 길이: 18
별점 점수(2022년 종료 

블로그 리뷰 수: 785
blog_review_qty_list 길이: 31
별점 점수(2022년 종료 예정): 4.51
stars_list 길이: 31
별점 평가 수(2022년 종료 예정): 390개 평점 524명 참여
stars_qty_list 길이: 31
방문자 평가 참여자수: 244
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '인테리어가 멋져요', '친절해요']
32번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 32
블로그 리뷰 수: 589
blog_review_qty_list 길이: 32
별점 점수(2022년 종료 예정): 4.39
stars_list 길이: 32
별점 평가 수(2022년 종료 예정): 716개 평점 942명 참여
stars_qty_list 길이: 32
방문자 평가 참여자수: 515
해당 플레이스 리뷰: ['인테리어가 멋져요']
해당 플레이스 리뷰: ['인테리어가 멋져요', '커피가 맛있어요']
해당 플레이스 리뷰: ['인테리어가 멋져요', '커피가 맛있어요', '디저트가 맛있어요']
해당 플레이스 리뷰: ['인테리어가 멋져요', '커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요']
해당 플레이스 리뷰: ['인테리어가 멋져요', '커피가 맛있어요', '디저트가 맛있어요', '음료가 맛있어요', '사진이 잘 나와요']
33번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 33
블로그 리뷰 수: 388
blog_review_qty_list 길이: 33
별점 

...

664번째 데이터 수집 시작
naver_map_type: 브런치
naver_map_type_list 길이: 664
블로그 리뷰 수: 848
blog_review_qty_list 길이: 664
별점 점수(2022년 종료 예정): 4.37
stars_list 길이: 664
별점 평가 수(2022년 종료 예정): 591개 평점 657명 참여
stars_qty_list 길이: 664
방문자 평가 참여자수: 264
해당 플레이스 리뷰: ['음식이 맛있어요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '특별한 메뉴가 있어요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '특별한 메뉴가 있어요', '재료가 신선해요']
해당 플레이스 리뷰: ['음식이 맛있어요', '인테리어가 멋져요', '특별한 메뉴가 있어요', '재료가 신선해요', '친절해요']
665번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 665
블로그 리뷰 수: 176
blog_review_qty_list 길이: 665
별점 점수(2022년 종료 예정): 4.43
stars_list 길이: 665
별점 평가 수(2022년 종료 예정): 2,883개 평점 2,554명 참여
stars_qty_list 길이: 665
방문자 평가 참여자수: 1,197
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '디저트가 맛있어요']
666번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_lis

679번째 데이터 수집 시작
naver_map_type: 만화방
naver_map_type_list 길이: 679
블로그 리뷰 수: 125
blog_review_qty_list 길이: 679
Element 요소 미존재
stars_list 길이: 679
Element 요소 미존재
stars_qty_list 길이: 679
방문자 평가 참여자수: 52
해당 플레이스 리뷰: ['친절해요']
해당 플레이스 리뷰: ['친절해요', '매장이 청결해요']
680번째 데이터 수집 시작
naver_map_type: 케이크전문
naver_map_type_list 길이: 680
블로그 리뷰 수: 147
blog_review_qty_list 길이: 680
별점 점수(2022년 종료 예정): 4.36
stars_list 길이: 680
별점 평가 수(2022년 종료 예정): 985개 평점 1,068명 참여
stars_qty_list 길이: 680
방문자 평가 참여자수: 393
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '음료가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '음료가 맛있어요', '친절해요', '매장이 청결해요']
681번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 681
블로그 리뷰 수: 222
blog_review_qty_list 길이: 681
별점 점수(2022년 종료 예정): 4.55
stars_list 길이: 681
별점 평가 수(2022년 종료 예정): 351개 평점 856명 참여
stars_qty_list 길이: 681
방문자 평가 참여자수: 1,044
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어

블로그 리뷰 수: 123
blog_review_qty_list 길이: 694
별점 점수(2022년 종료 예정): 4.44
stars_list 길이: 694
별점 평가 수(2022년 종료 예정): 2,986개 평점 2,706명 참여
stars_qty_list 길이: 694
방문자 평가 참여자수: 953
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요', '디저트가 맛있어요']
695번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 695
블로그 리뷰 수: 121
blog_review_qty_list 길이: 695
별점 점수(2022년 종료 예정): 4.51
stars_list 길이: 695
별점 평가 수(2022년 종료 예정): 139개 평점 179명 참여
stars_qty_list 길이: 695
방문자 평가 참여자수: 84
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '인테리어가 멋져요', '친절해요', '음료가 맛있어요', '디저트가 맛있어요']
696번째 데이터 수집 시작
naver_map_type: 케이크전문
naver_map_type_list 길이: 696
블로그 리뷰 수: 155
blog_review_qty_list 길이: 696
별점 점수(2022년 종료 예정): 4.4

709번째 데이터 수집 시작
naver_map_type: 빙수
naver_map_type_list 길이: 709
블로그 리뷰 수: 223
blog_review_qty_list 길이: 709
별점 점수(2022년 종료 예정): 4.27
stars_list 길이: 709
별점 평가 수(2022년 종료 예정): 1,130개 평점 1,474명 참여
stars_qty_list 길이: 709
방문자 평가 참여자수: 701
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '음료가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '커피가 맛있어요', '음료가 맛있어요', '특별한 메뉴가 있어요', '친절해요']
710번째 데이터 수집 시작
naver_map_type: 스터디카페
naver_map_type_list 길이: 710
블로그 리뷰 수: 361
blog_review_qty_list 길이: 710
별점 점수(2022년 종료 예정): 4.37
stars_list 길이: 710
별점 평가 수(2022년 종료 예정): 224개 평점 154명 참여
stars_qty_list 길이: 710
방문자 평가 참여자수 10명 미만인 장소
711번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 711
블로그 리뷰 수: 176
blog_review_qty_list 길이: 711
별점 점수(2022년 종료 예정): 4.53
stars_list 길이: 711
별점 평가 수(2022년 종료 예정): 3,658개 평점 2,955명 참여
stars_qty_list 길이: 711
방문자 평가 참여자수: 1,128
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있

724번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_list 길이: 724
블로그 리뷰 수: 32
blog_review_qty_list 길이: 724
별점 점수(2022년 종료 예정): 4.71
stars_list 길이: 724
별점 평가 수(2022년 종료 예정): 155개 평점 114명 참여
stars_qty_list 길이: 724
방문자 평가 참여자수: 36
해당 플레이스 리뷰: ['디저트가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요', '커피가 맛있어요']
해당 플레이스 리뷰: ['디저트가 맛있어요', '특별한 메뉴가 있어요', '친절해요', '커피가 맛있어요', '매장이 청결해요']
725번째 데이터 수집 시작
naver_map_type: 카페
naver_map_type_list 길이: 725
블로그 리뷰 수: 46
blog_review_qty_list 길이: 725
별점 점수(2022년 종료 예정): 4.33
stars_list 길이: 725
별점 평가 수(2022년 종료 예정): 966개 평점 1,010명 참여
stars_qty_list 길이: 725
방문자 평가 참여자수: 470
해당 플레이스 리뷰: ['커피가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '친절해요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '친절해요', '음료가 맛있어요']
해당 플레이스 리뷰: ['커피가 맛있어요', '디저트가 맛있어요', '친절해요', '음료가 맛있어요', '매장이 청결해요']
726번째 데이터 수집 시작
naver_map_type: 카페,디저트
naver_map_type_

In [19]:
driver.quit()
sub_driver.quit()

In [20]:
print(dict_review)

{'커피가 맛있어요': ['', '', '1408', '1333', '', '234', '1634', '149', '793', '', '514', '253', '739', '106', '265', '3', '', '249', '', '31', '90', '', '', '150', '47', '7', '', '411', '68', '191', '133', '222', '210', '89', '196', '128', '103', '658', '', '119', '330', '150', '456', '106', '907', '408', '305', '174', '985', '340', '703', '320', '285', '191', '55', '109', '556', '83', '', '507', '179', '', '48', '407', '', '655', '109', '159', '118', '119', '985', '100', '45', '72', '34', '985', '57', '204', '', '313', '90', '37', '11', '', '18', '16', '719', '1045', '565', '', '', '', '', '98', '', '181', '204', '107', '71', '153', '229', '', '148', '185', '107', '157', '', '', '72', '', '92', '125', '86', '379', '123', '337', '175', '47', '209', '158', '', '94', '12', '10', '68', '106', '28', '66', '80', '34', '2352', '', '', '', '314', '536', '194', '931', '525', '180', '89', '366', '284', '111', '283', '104', '309', '98', '76', '77', '189', '146', '', '231', '530', '', '59', '53', '68', 

In [21]:
print(len(dict_review['혼밥하기 좋아요']))

726


In [22]:
print(len(naver_map_type_list))

726


In [23]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

In [24]:
print(df2)

             id    place_name       address_name           phone 커피가 맛있어요  \
0    1974266862      카페노티드 안국       서울 종로구 재동 86  0507-1337-9973            
1       8574728    토즈모임센터 종로점   서울 종로구 종로2가 84-8     02-736-0061            
2     853028118       어니언 안국점    서울 종로구 계동 146-1   070-7543-2123     1408   
3     353512935    블루보틀 삼청 카페      서울 종로구 소격동 86       1533-6906     1333   
4    1779571274         안녕인사동   서울 종로구 관훈동 155-2    02-6954-2910            
..          ...           ...                ...             ...      ...   
721   649401592     스타벅스 구로NC     서울 구로구 구로동 573       1522-3232     1229   
722    27075938  스타벅스 현대디큐브2F    서울 구로구 신도림동 692       1522-3232      902   
723  1566212360   그녀의커피 카페드엘르   서울 구로구 오류동 6-140    02-6013-7750       11   
724  1893731799   투썸플레이스 개봉역점  서울 구로구 개봉동 202-13    02-2618-8133      249   
725   963590299          잉크커피  서울 구로구 구로동 222-12             NaN        7   

    디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요  ... 단체모임 하기 좋아요 특별한 날 가기 좋아요

In [25]:
#df2 내용 csv로 저장
#df2.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/sample.csv')
#df2.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구_카페_review.csv')
df2.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/서울특별시 카페 review.csv')

In [26]:
#df2_json = df2.to_json('/Users/dc/study_dir/recommendation/data/crawl/cafe/성동구_카페_review.json', orient='records', indent=4, force_ascii=False)
#df2_json = df2.to_json('/Users/dc/practice/recommendation/recomm_drf/data/crawl/cafe/성동구_review.json', orient='records', indent=4, force_ascii=False)
# print(df2_json)
df2_json = df2.to_json('/Users/dc/study_dir/recommendation/data/crawl/cafe/서울특별시 카페 review.json', orient='records', indent=4, force_ascii=False)

In [2]:
# test(2022.10.25(화))
import os
# import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import pandas as pd
import numpy as np
import json
import time
# from config.settings.base import CAFE, DATA_DIR, CSV_DATA_PATH
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm.notebook import tqdm
import re
from tqdm import *
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#df3 = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '성동구_review_rating.csv'), sep=',')
#df3 = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '성동구_카페_rating.csv'), sep=',')
#df3 = pd.read_csv(os.path.join('/Users/dc/study_dir/recommendation/data/csv/cafe', '서울특별시 카페 rating.csv'), sep=',')
df3 = pd.read_csv(os.path.join('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe', '서울특별시 카페 rating.csv'), sep=',')
df3 = df3.iloc[:, 1:4]
df3

,place_name,review,rating
0,카페노티드 안국,커피가 맛있어요,NaN
1,카페노티드 안국,디저트가 맛있어요,3151.0
2,카페노티드 안국,빵이 맛있어요,NaN
3,카페노티드 안국,친절해요,884.0
4,카페노티드 안국,인테리어가 멋져요,348.0
...,...,...,...
16693,잉크커피,단체모임 하기 좋아요,NaN
16694,잉크커피,특별한 날 가기 좋아요,NaN
16695,잉크커피,재료가 신선해요,NaN
16696,잉크커피,양이 많아요,NaN


In [4]:
df4 = df3.dropna()
df4

,place_name,review,rating
1,카페노티드 안국,디저트가 맛있어요,3151.0
3,카페노티드 안국,친절해요,884.0
4,카페노티드 안국,인테리어가 멋져요,348.0
5,카페노티드 안국,매장이 청결해요,434.0
9,카페노티드 안국,특별한 메뉴가 있어요,1029.0
...,...,...,...
16675,잉크커피,커피가 맛있어요,7.0
16678,잉크커피,친절해요,5.0
16682,잉크커피,음료가 맛있어요,4.0
16686,잉크커피,가성비가 좋아요,3.0


In [15]:
visitor_review_list = ['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                       '양이 많아요', '혼밥하기 좋아요']
len(visitor_review_list)

23

In [16]:
# review_values = df4['review'].values
# review_list = review_values.tolist()
# review_list = list(set(review_list))
# review_list

In [17]:
# for i in visitor_review_list:
#     if i not in review_list:
#         print(i)

In [18]:
few_reviews = []
for i in visitor_review_list:
    review_cnt = len(df4.loc[df4['review'] == i])
    print(f'{i} 리뷰 조회 갯수: {review_cnt}')
    if review_cnt < 2:
        few_reviews.append(i)

커피가 맛있어요 리뷰 조회 갯수: 576
디저트가 맛있어요 리뷰 조회 갯수: 468
빵이 맛있어요 리뷰 조회 갯수: 24
친절해요 리뷰 조회 갯수: 545
인테리어가 멋져요 리뷰 조회 갯수: 363
매장이 청결해요 리뷰 조회 갯수: 288
뷰가 좋아요 리뷰 조회 갯수: 66
음료가 맛있어요 리뷰 조회 갯수: 510
사진이 잘 나와요 리뷰 조회 갯수: 55
특별한 메뉴가 있어요 리뷰 조회 갯수: 172
대화하기 좋아요 리뷰 조회 갯수: 97
가성비가 좋아요 리뷰 조회 갯수: 37
집중하기 좋아요 리뷰 조회 갯수: 23
화장실이 깨끗해요 리뷰 조회 갯수: 0
주차하기 편해요 리뷰 조회 갯수: 10
좌석이 편해요 리뷰 조회 갯수: 13
음식이 맛있어요 리뷰 조회 갯수: 47
매장이 넓어요 리뷰 조회 갯수: 10
단체모임 하기 좋아요 리뷰 조회 갯수: 6
특별한 날 가기 좋아요 리뷰 조회 갯수: 0
재료가 신선해요 리뷰 조회 갯수: 22
양이 많아요 리뷰 조회 갯수: 5
혼밥하기 좋아요 리뷰 조회 갯수: 3


In [19]:
print(few_reviews)

['화장실이 깨끗해요', '특별한 날 가기 좋아요']


In [25]:
df5 = df4.copy()
for i in few_reviews:
    #df5 = df5[df5['review']!= i]
    #print(i)
    
    #idx = df5[df5['review'] == i].index
    idx = df5[df5['review'].str.contains(i)].index
    print(idx)
    df5.drop(idx, inplace=True)
    df5.reset_index(drop=True, inplace=True)
    
df5
#df5['review'] != '화장실이 깨끗해요'

Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


,place_name,review,rating
1,카페노티드 안국,디저트가 맛있어요,3151.0
3,카페노티드 안국,친절해요,884.0
4,카페노티드 안국,인테리어가 멋져요,348.0
5,카페노티드 안국,매장이 청결해요,434.0
9,카페노티드 안국,특별한 메뉴가 있어요,1029.0
...,...,...,...
16675,잉크커피,커피가 맛있어요,7.0
16678,잉크커피,친절해요,5.0
16682,잉크커피,음료가 맛있어요,4.0
16686,잉크커피,가성비가 좋아요,3.0


In [26]:
df5.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/서울특별시 카페 전처리1.csv')

In [28]:
# dup = df5.duplicated(['review', 'place_name'])
# dup

In [132]:
#df6 = pd.concat([df5, dup], axis=1)
#df6.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구 카페 정보_전처리3.csv')

In [90]:
#df5.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구 카페 정보_전처리1.csv')

In [104]:
#df5 = df5.drop_duplicates()
#df5

# df6 = df5.groupby(['review', 'place_name'], as_index=False).mean()
# df6.to_csv('/Users/dc/study_dir/recommendation/data/csv/cafe/성동구 카페 정보_전처리2.csv')

In [29]:
x = df5.copy()
y = df5['review']

# 테스트 샘플 데이터에서 사용
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# 실제 대용량 데이터에서 사용
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [52]:
x_test

,place_name,review,rating
1452,더피터커피,친절해요,194.0
5018,카페몰로,인테리어가 멋져요,43.0
13275,오츠커피 마포점,인테리어가 멋져요,48.0
15647,스타벅스 마곡나루공원,음료가 맛있어요,267.0
12729,시간이머무는홍차가게,대화하기 좋아요,18.0
...,...,...,...
4609,열매제과점,특별한 메뉴가 있어요,22.0
3634,카페오롯,커피가 맛있어요,68.0
11618,탐나도다,친절해요,11.0
5592,37.5 답십리점,친절해요,16.0


In [49]:
#x_train = x_train.groupby(['review', 'place_name'])

In [50]:
#x_train.shape

In [31]:
#rating_matrix = pd.pivot_table(x_train, index='review', columns='place_name', values = 'rating', aggfunc='mean')
rating_matrix = pd.pivot_table(x_train, index='review', columns='place_name', values = 'rating')
rating_matrix

place_name,153도씨,1인1잔,1일1잔,213점,33아파트먼트,37.5 답십리점,502세컨즈카페 노원점,5TO7,823m,8번가,...,호랑이,호밀밭,호텔세느장,호텔수선화,홍만당 명동본점,훔볼트,휴가,히어로스터 신도림점,힐링앤툰,힙스터커피
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,145.0,NaN,NaN,NaN,15.0,NaN,NaN,NaN,4.0,NaN
단체모임 하기 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대화하기 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,11.0
디저트가 맛있어요,180.0,234.0,9.0,34.0,9.0,NaN,NaN,NaN,25.0,86.0,...,NaN,406.0,88.0,NaN,228.0,NaN,NaN,31.0,NaN,NaN
매장이 넓어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
매장이 청결해요,56.0,NaN,NaN,17.0,NaN,20.0,NaN,NaN,NaN,NaN,...,NaN,90.0,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN
뷰가 좋아요,NaN,615.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
빵이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
사진이 잘 나와요,NaN,270.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,86.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# rating_matrix = x_train.pivot(index='review', columns='place_name', values='rating')
# rating_matrix

In [33]:
matrix_dummy = rating_matrix.copy().fillna(0)
review_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
review_similarity = pd.DataFrame(review_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

In [34]:
review_similarity

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,음식이 맛있어요,인테리어가 멋져요,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요,혼밥하기 좋아요
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,1.000000,0.008965,0.000025,0.014489,0.000460,0.018150,0.000000,0.021818,0.000000,0.592677,...,0.330408,0.004828,0.405152,0.000021,0.000000,0.000000,0.039548,0.052341,0.018792,0.507228
단체모임 하기 좋아요,0.008965,1.000000,0.000000,0.000000,0.601744,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.089461,0.000000,0.000000,0.000000,0.000000,0.001955,0.000000,0.000000,0.000000
대화하기 좋아요,0.000025,0.000000,1.000000,0.075366,0.000000,0.104845,0.023796,0.014762,0.008319,0.000000,...,0.000099,0.173493,0.000000,0.368667,0.029417,0.135379,0.043384,0.174656,0.005652,0.000000
디저트가 맛있어요,0.014489,0.000000,0.075366,1.000000,0.000000,0.339110,0.071471,0.000000,0.318493,0.000000,...,0.000000,0.465017,0.000000,0.017188,0.000000,0.021714,0.530765,0.348393,0.656172,0.000000
매장이 넓어요,0.000460,0.601744,0.000000,0.000000,1.000000,0.042916,0.021989,0.000000,0.000000,0.093696,...,0.257980,0.131462,0.016665,0.000000,0.000000,0.000000,0.010509,0.000000,0.011831,0.000000
매장이 청결해요,0.018150,0.000000,0.104845,0.339110,0.042916,1.000000,0.028580,0.118800,0.000000,0.000602,...,0.038160,0.170706,0.020136,0.142366,0.119681,0.009515,0.628728,0.641602,0.149190,0.000000
뷰가 좋아요,0.000000,0.000000,0.023796,0.071471,0.021989,0.028580,1.000000,0.140428,0.233374,0.000000,...,0.014507,0.179106,0.004085,0.000000,0.025398,0.010255,0.074500,0.167755,0.007206,0.000000
빵이 맛있어요,0.021818,0.000000,0.014762,0.000000,0.000000,0.118800,0.140428,1.000000,0.000349,0.000000,...,0.000000,0.035617,0.000000,0.000000,0.000000,0.000000,0.114365,0.061254,0.111127,0.000000
사진이 잘 나와요,0.000000,0.000000,0.008319,0.318493,0.000000,0.000000,0.233374,0.000349,1.000000,0.000000,...,0.000000,0.563856,0.000000,0.000036,0.000000,0.000000,0.172885,0.080890,0.010508,0.000000


In [35]:
#### 플레이스 전반에 대한 review 경향을 고려한 함수 ####. axis=1 은 행 기준으로 평균을 구함(여기서 행은 review)
rating_mean = rating_matrix.mean(axis=1)
rating_mean

review
가성비가 좋아요        52.857143
단체모임 하기 좋아요     87.000000
대화하기 좋아요        72.424658
디저트가 맛있어요      191.831429
매장이 넓어요        128.875000
매장이 청결해요       129.930556
뷰가 좋아요         143.380000
빵이 맛있어요        443.222222
사진이 잘 나와요       94.097561
양이 많아요          97.750000
음료가 맛있어요       116.900000
음식이 맛있어요       187.514286
인테리어가 멋져요      113.863469
재료가 신선해요       131.187500
좌석이 편해요         58.700000
주차하기 편해요       117.125000
집중하기 좋아요        80.941176
친절해요           126.899510
커피가 맛있어요       304.174419
특별한 메뉴가 있어요    114.765625
혼밥하기 좋아요       272.500000
dtype: float64

In [36]:
# 각 플레이스에 대한 리뷰 전체적인 평점
rating_matrix.T

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,음식이 맛있어요,인테리어가 멋져요,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요,혼밥하기 좋아요
place_name,,,,,,,,,,,,,,,,,,,,,
153도씨,NaN,NaN,NaN,180.0,NaN,56.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1인1잔,NaN,NaN,NaN,234.0,NaN,NaN,615.0,NaN,270.0,NaN,...,NaN,308.0,NaN,NaN,NaN,NaN,NaN,416.0,NaN,NaN
1일1잔,3.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN
213점,NaN,NaN,NaN,34.0,NaN,17.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.0,36.0,NaN,NaN
33아파트먼트,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,15.0,NaN,NaN,NaN,NaN,18.0,47.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
훔볼트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,128.0,NaN,NaN,NaN,NaN,NaN,239.0,NaN,NaN
휴가,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,86.0,NaN,NaN,NaN,NaN,52.0,76.0,NaN,NaN
히어로스터 신도림점,NaN,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,45.0,102.0,NaN,NaN


In [37]:
# 각 플레이스 평점(rating_matrix.T)에 대한 리뷰별 경향(rating_mean. 리뷰의 평점평균)을 고려하여 플레이스 평점평균 '편차(bias)' 계산
rating_matrix.T - rating_mean

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,음식이 맛있어요,인테리어가 멋져요,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요,혼밥하기 좋아요
place_name,,,,,,,,,,,,,,,,,,,,,
153도씨,NaN,NaN,NaN,-11.831429,NaN,-73.930556,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1인1잔,NaN,NaN,NaN,42.168571,NaN,NaN,471.62,NaN,175.902439,NaN,...,NaN,194.136531,NaN,NaN,NaN,NaN,NaN,111.825581,NaN,NaN
1일1잔,-49.857143,NaN,NaN,-182.831429,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-119.89951,NaN,NaN,NaN
213점,NaN,NaN,NaN,-157.831429,NaN,-112.930556,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-91.89951,-268.174419,NaN,NaN
33아파트먼트,NaN,NaN,NaN,-182.831429,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-98.863469,NaN,NaN,NaN,NaN,-108.89951,-257.174419,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
훔볼트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,14.136531,NaN,NaN,NaN,NaN,NaN,-65.174419,NaN,NaN
휴가,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-27.863469,NaN,NaN,NaN,NaN,-74.89951,-228.174419,NaN,NaN
히어로스터 신도림점,NaN,NaN,NaN,-160.831429,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-81.89951,-202.174419,NaN,NaN


In [38]:
rating_bias = (rating_matrix.T - rating_mean).T
rating_bias

place_name,153도씨,1인1잔,1일1잔,213점,33아파트먼트,37.5 답십리점,502세컨즈카페 노원점,5TO7,823m,8번가,...,호랑이,호밀밭,호텔세느장,호텔수선화,홍만당 명동본점,훔볼트,휴가,히어로스터 신도림점,힐링앤툰,힙스터커피
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,NaN,NaN,-49.857143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.142857,NaN,NaN,NaN,-37.857143,NaN,NaN,NaN,-48.857143,NaN
단체모임 하기 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
대화하기 좋아요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-50.424658,NaN,NaN,NaN,NaN,NaN,-61.424658
디저트가 맛있어요,-11.831429,42.168571,-182.831429,-157.831429,-182.831429,NaN,NaN,NaN,-166.831429,-105.831429,...,NaN,214.168571,-103.831429,NaN,36.168571,NaN,NaN,-160.831429,NaN,NaN
매장이 넓어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
매장이 청결해요,-73.930556,NaN,NaN,-112.930556,NaN,-109.930556,NaN,NaN,NaN,NaN,...,NaN,-39.930556,NaN,NaN,-112.930556,NaN,NaN,NaN,NaN,NaN
뷰가 좋아요,NaN,471.620000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
빵이 맛있어요,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
사진이 잘 나와요,NaN,175.902439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-8.097561,-77.097561,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [40]:
# 모델별 RMSE를 계산하는 함수(test set 활용하여 계산)
def score(model, neigbor_size=0):
    # 테스트 데이터의 review와 place_name 간 pair를 맞춰 튜플형 원소 리스트 데이터를 만듬
    id_pairs = zip(x_test['review'], x_test['place_name'])
    # 모든 review-플레이스 짝에 대해서 주어진 예측모델에 의해 예측값 계산 및 리스트형 데이터 생성
    y_pred = np.array([model(review, place, neigbor_size) for (review, place) in id_pairs])
    # 실제 평점값
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [34]:
# id_pairs_list = list(zip(x_test['review'], x_test['place_name']))
# print(id_pairs_list)

[('특별한 날 가기 좋아요', '카페키이로'), ('디저트가 맛있어요', '스태픽스'), ('단체모임 하기 좋아요', '내자상회'), ('주차하기 편해요', '어니언 안국점'), ('단체모임 하기 좋아요', '서울서 둘째로잘하는집'), ('친절해요', '내자상회'), ('음료가 맛있어요', '토즈모임센터 종로점'), ('가성비가 좋아요', '개뿔'), ('화장실이 깨끗해요', '카페 레이어드 안국점'), ('재료가 신선해요', '테라로사 광화문점'), ('친절해요', '산모퉁이'), ('재료가 신선해요', '라카페갤러리'), ('디저트가 맛있어요', '대충유원지 인왕산'), ('대화하기 좋아요', '카페노티드 안국'), ('뷰가 좋아요', '개뿔'), ('화장실이 깨끗해요', '카페노티드 안국'), ('집중하기 좋아요', '차마시는 뜰'), ('인테리어가 멋져요', '프릳츠 원서점'), ('화장실이 깨끗해요', '토즈모임센터 광화문점'), ('뷰가 좋아요', '도트블랭킷'), ('특별한 메뉴가 있어요', '개뿔'), ('사진이 잘 나와요', '산모퉁이'), ('친절해요', '슬로우파크'), ('디저트가 맛있어요', '슬로우파크'), ('특별한 날 가기 좋아요', '라카페갤러리'), ('양이 많아요', '토즈모임센터 광화문점'), ('주차하기 편해요', '카페키이로'), ('화장실이 깨끗해요', '대충유원지 인왕산'), ('음료가 맛있어요', 'H라운지'), ('집중하기 좋아요', '카페아르크'), ('특별한 메뉴가 있어요', '카페 하이웨이스트 익선점'), ('혼밥하기 좋아요', '개뿔'), ('커피가 맛있어요', '어니언 안국점'), ('음식이 맛있어요', '안녕인사동'), ('빵이 맛있어요', '청수당'), ('집중하기 좋아요', '카페노티드 안국'), ('음료가 맛있어요', '도트블랭킷'), ('빵이 맛있어요', '아키비스트'), ('주차하기 편해요', '서울서 둘째로잘하는집'), ('매장이 넓어요', '아키비스트'), ('음식이 맛있어요', '카페노티

In [41]:
# rating_matrix > 0 을 통해 나온 True 값들을 numpy array로 만들고, 이렇게 나온 True, False 구분값들을 1과 0의 binary 값으로 변환
rating_binary_1 = np.array(rating_matrix > 0).astype(float)
rating_binary_2 = rating_binary_1.T

counts = np.dot(rating_binary_1, rating_binary_2)
counts = pd.DataFrame(counts,
                      index=rating_matrix.index,
                      columns=rating_matrix.index).fillna(0)

In [42]:
# 각 review 평가가 되어있는 플레이스의 갯수를 나타내기 (대각선 = 각 review 평가가 되어있는 플레이스의 갯수, 다른 값들은 review간 공통으로 평가되어있는 플레이스의 갯수)
counts

review,가성비가 좋아요,단체모임 하기 좋아요,대화하기 좋아요,디저트가 맛있어요,매장이 넓어요,매장이 청결해요,뷰가 좋아요,빵이 맛있어요,사진이 잘 나와요,양이 많아요,...,음식이 맛있어요,인테리어가 멋져요,재료가 신선해요,좌석이 편해요,주차하기 편해요,집중하기 좋아요,친절해요,커피가 맛있어요,특별한 메뉴가 있어요,혼밥하기 좋아요
review,,,,,,,,,,,,,,,,,,,,,
가성비가 좋아요,28.0,1.0,1.0,6.0,1.0,6.0,0.0,2.0,0.0,2.0,...,6.0,5.0,4.0,1.0,0.0,0.0,21.0,8.0,5.0,1.0
단체모임 하기 좋아요,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
대화하기 좋아요,1.0,0.0,73.0,31.0,0.0,11.0,3.0,1.0,2.0,0.0,...,1.0,29.0,0.0,3.0,2.0,4.0,25.0,47.0,3.0,0.0
디저트가 맛있어요,6.0,0.0,31.0,350.0,0.0,100.0,16.0,0.0,21.0,0.0,...,0.0,129.0,0.0,4.0,0.0,4.0,192.0,240.0,65.0,0.0
매장이 넓어요,1.0,1.0,0.0,0.0,8.0,3.0,3.0,0.0,0.0,1.0,...,6.0,5.0,1.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0
매장이 청결해요,6.0,0.0,11.0,100.0,3.0,216.0,6.0,9.0,0.0,1.0,...,13.0,58.0,5.0,2.0,1.0,2.0,142.0,118.0,31.0,0.0
뷰가 좋아요,0.0,0.0,3.0,16.0,3.0,6.0,50.0,3.0,9.0,0.0,...,3.0,29.0,1.0,0.0,3.0,2.0,15.0,32.0,3.0,0.0
빵이 맛있어요,2.0,0.0,1.0,0.0,0.0,9.0,3.0,18.0,1.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,13.0,7.0,8.0,0.0
사진이 잘 나와요,0.0,0.0,2.0,21.0,0.0,0.0,9.0,1.0,41.0,0.0,...,0.0,29.0,0.0,1.0,0.0,0.0,7.0,24.0,5.0,0.0


In [43]:
SIG_LEVEL = 3
MIN_RATINGS = 2

In [45]:
def CF_knn_bias_sig(review, place, neighbor_size=0):
    if place in rating_bias.columns:
        # input으로 들어온 review와 다른 review 간의 유사도 추출
        sim_scores = review_similarity[review].copy()
        # 해당 플레이스에 대한 review 평가 경향고려한(rating_bias 데이터프레임) 평점 정보 추출
        place_ratings = rating_bias[place].copy()
        # 플레이스 평가에 등록되어있지 않은 review인지 추출
        no_rating = place_ratings.isnull() # True, False
        # review가 공통으로 등록되어있는 플레이스의 갯수를 할당
        common_counts = counts[review]
        
        low_significance = common_counts < SIG_LEVEL   # SIG_LEVEL = 미리 설정한 숫자 보다 작은 review는 True로 변환
        # 플레이스 평가등록되지 않은 review or 지정한 SIG_LEVEL보다 적게 공통의 플레이스를 평가한 review의 index 추출
        none_rating_idx = place_ratings[no_rating | low_significance].index

        place_ratings = place_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:
            # review 평가 경향을 반영한 가중평균 (플레이스점수*가중치(review간 유사도) / 가중치의 합)
            prediction = np.dot(sim_scores, place_ratings) / sim_scores.sum()
            # '가중평균(위의 prediction)' + '플레이스 전반에 대한 해당 review 평균 평점' 하여 reveiw 카테고리의 최종 플레이스 예상 평점 구함 
            prediction += rating_mean[review]
        else: 
            if len(sim_scores) > MIN_RATINGS:     # MIN_RATINGS = 미리 설정한 최소 review보다 현재 플레이스를 평가한 유사한 review가 큰 경우만
                print('=============='*5)
                print('MIN_RATINGS 보다 큰 상황')
                print('=============='*5)
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                place_ratings = np.array(place_ratings)
                # 유사도 오름차순 정렬후 user index 추출
                review_idx = np.argsort(sim_scores)
                # 입력한 neighbor_size 만큼 높은 유사도 정보 추출
                sim_scores = sim_scores[review_idx][-neighbor_size:]
                # 평점 정보 추출
                place_ratings = place_ratings[review_idx][-neighbor_size:]
                # review 평가 경향을 반영한 가중평균 (플레이스점수*가중치(review간 유사도) / 가중치의 합)
                prediction = np.dot(sim_scores, place_ratings) / sim_scores.sum()
                # '가중평균(위의 prediction)' + '플레이스 전반에 대한 해당 review 평균 평점' 하여 reveiw 카테고리의 최종 플레이스 예상 평점 구함 
                prediction += rating_mean[review]
            else:
                print('=============='*5)
                print('MIN_RATINGS 보다 작은 상황')
                print('=============='*5)
                # 원래 review의 평균 평점을 예상값으로 지정
                prediction = rating_mean[review]
    else:   # place에 대항하는 플레이스 정보가 rating_bias에 없는 경우
        # 원래 review의 평균 평점을 예상값으로 지정
        prediction = rating_mean[review]  # 강의 원본

    return prediction

In [46]:
CF_knn_bias_sig('커피가 맛있어요', '에이치큐브', 3)

MIN_RATINGS 보다 큰 상황


228.2813365679528

In [54]:
# SIG_LEVEL = 3
# MIN_RATINGS = 3   # 1이상
score(CF_knn_bias_sig, neigbor_size=3)

MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN

MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_

173.73140137351456

In [75]:
# 원본
# def recom_place(review, n_items, neighbor_size=0):
#     # review가 등록되어 있는 플레이스 정보 추출
#     review_place = rating_matrix.loc[review].copy()
#     print(f'review_place: {review_place}')
#     print('=============='*5)
    
#     for place in rating_matrix.columns:
#         # 해당 플레이스에 대한 review가 이미 존재하는 경우, 추천 리스트에서 제외???(장소 추천시스템에선 불필요...?)
#         if pd.notnull(review_place.loc[place]):
#             review_place.loc[place] = 0
#         else:
#             review_place.loc[place] = CF_knn_bias_sig(review, place, neighbor_size)
            
#     place_sort = review_place.sort_values(ascending=False)[:n_items]
#     print(f'place_sort: {place_sort}')
#     recom_places = movies.loc[place_sort.index]
#     recommendations = recom_places['title']
#     return recommendations

In [50]:
# 테스트 중
def recom_place(review, n_items, neighbor_size=0):
    # review가 등록되어 있는 플레이스 정보 추출
    review_place = rating_matrix.loc[review].copy()
    print(f'review_place: {review_place}')
    print('=============='*5)
    
    for place in rating_matrix.columns:
        review_place.loc[place] = CF_knn_bias_sig(review, place, neighbor_size)
        
    print(f'review_place 최종: {review_place}')
            
    place_sort = review_place.sort_values(ascending=False)[:n_items]
    print('=============='*5)
    print(f'place_sort: {place_sort}')
#     recom_places = movies.loc[place_sort.index]
#     recommendations = recom_places['title']
#     return recommendations

In [53]:
recom_place('인테리어가 멋져요', 3, 3)

review_place: place_name
153도씨           NaN
1인1잔          308.0
1일1잔            NaN
213점            NaN
33아파트먼트        15.0
              ...  
훔볼트           128.0
휴가             86.0
히어로스터 신도림점      NaN
힐링앤툰            NaN
힙스터커피           NaN
Name: 인테리어가 멋져요, Length: 689, dtype: float64
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 작은 상황
MIN_RATINGS 보다 큰 상황
MIN_RATINGS 보다 큰 상황
MIN_

In [7]:
# ratings = df3.groupby(['place_name'])[['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
#             a                           '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
#                                        '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
#                                        '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
#                                        '양이 많아요', '혼밥하기 좋아요']].sum()

# ratings
ratings = df3.pivot(index=['place_name'],
         columns=['place_name'],
         values=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                 '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                 '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                 '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                 '양이 많아요', '혼밥하기 좋아요'])

ratings

ValueError: The name place_name occurs multiple times, use a level number

In [161]:
df3 = df2[['place_name', '커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
                       '양이 많아요', '혼밥하기 좋아요']]

In [162]:
place_rating = df3.copy()
place_rating = place_rating.replace('', np.nan)
place_rating = place_rating.fillna(0)
# place_rating = place_rating.T
print(place_rating)
place_rating.to_csv('/Users/dc/practice/recommendation/recomm_drf/data/csv/cafe/sample2.csv')

       place_name 커피가 맛있어요 디저트가 맛있어요 빵이 맛있어요 친절해요 인테리어가 멋져요 매장이 청결해요 뷰가 좋아요  \
0      토즈모임센터 종로점        0         0       0    0         0        0      0   
1        카페노티드 안국        0      3102       0  871       333      423      0   
2         어니언 안국점     1396      2231       0  983      1436        0      0   
3      블루보틀 삼청 카페     1323         0       0  536       434      428    430   
4            스태픽스      230       166       0    0       148        0    323   
5           아키비스트     1618       437       0    0       356        0      0   
6           안녕인사동        0         0       0    0         0        0      0   
7            산모퉁이      144         0       0    0        72        0    324   
8         포비 광화문점      784         0    1580  509         0      265      0   
9          차마시는 뜰        0         0       0  219       164        0    192   
10      테라로사 광화문점      513       123       0    0       192      124      0   
11        프릳츠 원서점      712       455       0    0   

In [139]:
# rating = place_rating.groupby(['place_name'])[['커피가 맛있어요']].sum()
# rating = place_rating.groupby(['place_name'])[['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
#                        '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
#                        '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
#                        '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요'
#                        '양이 많아요', '혼밥하기 좋아요']].sum()


# rating

In [163]:
print(place_rating.columns[1:])
review_list = place_rating.columns[1:]
print(len(review_list))

Index(['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요',
       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요',
       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', '음식이 맛있어요',
       '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요양이 많아요', '혼밥하기 좋아요'],
      dtype='object')
22


In [164]:
rating_matrix = place_rating.pivot_table(index=['place_name'], 
                                   columns=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                                       '양이 많아요', '혼밥하기 좋아요'], 
                                   values=['커피가 맛있어요', '디저트가 맛있어요', '빵이 맛있어요', '친절해요', '인테리어가 멋져요', '매장이 청결해요', 
                                       '뷰가 좋아요', '음료가 맛있어요', '사진이 잘 나와요', '특별한 메뉴가 있어요', '대화하기 좋아요', 
                                       '가성비가 좋아요', '집중하기 좋아요', '화장실이 깨끗해요', '주차하기 편해요', '좌석이 편해요', 
                                       '음식이 맛있어요', '매장이 넓어요', '단체모임 하기 좋아요', '특별한 날 가기 좋아요', '재료가 신선해요',
                                       '양이 많아요', '혼밥하기 좋아요'],
                                  aggfunc='sum')


In [165]:
rating_matrix

place_name
H라운지
개뿔
나무사이로
내자상회
대충유원지 인왕산
도트블랭킷
라카페갤러리
부빙
블루보틀 삼청 카페
산모퉁이


In [159]:
rating_matrix = rating_matrix.stack()

ValueError: Must pass non-zero number of levels/codes

In [150]:
rating_matrix

place_name
H라운지
개뿔
나무사이로
내자상회
대충유원지 인왕산
도트블랭킷
라카페갤러리
부빙
블루보틀 삼청 카페
산모퉁이


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [86]:
x = place_rating.copy()
y = place_rating.iloc[0].to_list()
print(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

['토즈모임센터 종로점', '카페노티드 안국', '어니언 안국점', '블루보틀 삼청 카페', '스태픽스', '아키비스트', '안녕인사동', '산모퉁이', '포비 광화문점', '차마시는 뜰', '테라로사 광화문점', '프릳츠 원서점', '카페 레이어드 안국점', '나무사이로', '포비브라이트 광화문점', 'H라운지', '토즈모임센터 광화문점', '개뿔', '청수당', '부빙', '카페키이로', '도트블랭킷', '카페아르크', '대충유원지 인왕산', '카페 하이웨이스트 익선점', '라카페갤러리', '서울서 둘째로잘하는집', '슬로우파크', '내자상회']


In [ ]:
# 셀레니움 이용

for i, url in enumerate(tqdm_notebook(df['naver_map_url'])):
    driver.get(url)
    sub_driver.get(url+"/review/visitor")
    time.sleep(3.0)
#     driver.implicitly_wait(10)
#     sub_driver.implicitly_wait(10)

    # driver
    # 네이버 지도의 유형 분류
    try:
        naver_map_type = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#_title > span.DJJvD"))
        )
    except NoSuchElementException:
        print('Element 요소 미존재')
        naver_map_type_list.append('null')
    else:
        driver.implicitly_wait(5)
        print(f'naver_map_type: {naver_map_type.text}')
        naver_map_type_list.append(naver_map_type.text)
        driver.implicitly_wait(5)
    
#     # 네이버 지도의 유형 분류
#     naver_map_type = driver.find_element(By.CSS_SELECTOR, "#_title > span.DJJvD").text
#     print(f'naver_map_type: {naver_map_type}')
#     naver_map_type_list.append(naver_map_type)
#     #time.sleep(2.0)
#     driver.implicitly_wait(5)

    # 블로그 리뷰 수
    try:
#         blog_review_qty = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em"))
#         )
        blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(3) > a > em").text
        if not blog_review_qty:
            print('blog_review_qty ELEMNT does not exist & try other ELEMENT')
            driver.implicitly_wait(5)
#             blog_review_qty = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em"))
#             )
            blog_review_qty = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span:nth-child(2) > a > em").text
    except NoSuchElementException:
        print('Element 요소 미존재')
        blog_review_qty_list.append('null')
    else:
        driver.implicitly_wait(5)
        print(f'블로그 리뷰 수: {blog_review_qty}')
        blog_review_qty_list.append(blog_review_qty)
        #time.sleep(2.0)
        driver.implicitly_wait(5)

    # 별점 점수(2022년 종료 예정)
    try:
#         star_review_stars = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em"))
#         )
        star_review_stars = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm > div.dAsGb > span.PXMot.LXIwF > em").text
        driver.implicitly_wait(5)
        print(f'별점 점수(2022년 종료 예정): {star_review_stars}')
        stars_list.append(star_review_stars)
        #time.sleep(2.0)
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_list.append('null')



    # sub_driver
    
    # 별점 평가 수(2022년 종료 예정)  //*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]
    try:
#         star_review_qty = WebDriverWait(sub_driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]'))
#         )
        star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[1]/div/div/div[3]/span[2]').text
        if not star_review_qty:
            print('star_review_qty ELEMENT does not exist & try other ELEMENT')
            sub_driver.implicitly_wait(5)
#             star_review_qty = WebDriverWait(sub_driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]'))
#             )          
            star_review_qty = sub_driver.find_element(By.XPATH, r'//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/span[2]').text
    except NoSuchElementException:
        print('Element 요소 미존재')
        stars_qty_list.append('null')
    else:
        sub_driver.implicitly_wait(5)
        print(f'별점 평가 수(2022년 종료 예정): {star_review_qty}')
        stars_qty_list.append(star_review_qty)
        #time.sleep(2.0)


    # 방문자 평가 참여자수
    try:
        visitor_review_qty_element = WebDriverWait(sub_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span"))
        )
        visitor_review_qty = visitor_review_qty_element.text
        #visitor_review_qty = sub_driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div._Wmab > span").text
        visitor_review_qty = re.sub('[\(\)]', '', visitor_review_qty)
        visitor_review_qty = re.sub('명 참여', '', visitor_review_qty)
        sub_driver.implicitly_wait(5)
        print(f'방문자 평가 참여자수: {visitor_review_qty}')
        visitor_review_qty_list.append(visitor_review_qty)
        #time.sleep(2.0)
    except:
        print(f'방문자 평가 참여자수 10명 미만인 장소')
        visitor_review_qty = '9'
        visitor_review_qty_list.append(visitor_review_qty)


    # 방문자 호평 부분   # k2tmh   ,   uNsI9
    # visitor_eval_list = sub_driver.find_elements(By.CLASS_NAME, "k2tmh")
    # print(f'방문자 호평 부분: {visitor_eval_list}')

    try:
        visitor_eval_list_element = WebDriverWait(sub_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul"))
        )
        visitor_eval_list = visitor_eval_list_element.text
        #visitor_eval_list = sub_driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.mdJ86 > div > div > div.k2tmh > ul")
        sub_driver.implicitly_wait(5)
        print(f'방문자 호평 부분: {visitor_eval_list}')
        #sub_driver.implicitly_wait(5)
        
        for visitor_eval in visitor_eval_list[:5]:
            print('===========================')
            # 평가 전체 내역
            evaluation = visitor_eval.text
            print(f'evaluation: {evaluation}')
            print('===========================')
            print('for문을 통해 평가 내역 추출')
            evaluation = evaluation.split('\n')
            
            exist_review = []
            
            for i in range(0, 14, 3):
                print(evaluation[i])
                text_evaluation = re.sub('["]', '', evaluation[i])
                print('========'*5)
                print(text_evaluation)

                choose_evaluation = evaluation[i+2]
                print(f'choose_evaluation: {choose_evaluation}')
                print('========'*5)
                
                if text_evaluation in visitor_review_list:
                    dict_review[text_evaluation].append(choose_evaluation)
                    exist_review.append(text_evaluation)

#                 if text_evaluation in visitor_review_list:
#                     dict_review[text_evaluation].append(choose_evaluation)
            for review in visitor_review_list:
                if review not in exist_review:
                    dict_review[review].append('')
        #time.sleep(3.0)
    except:
        for i in visitor_review_list:
            dict_review[i].append('')


#     naver_map_type_list.append(naver_map_type)
#     blog_review_qty_list.append(blog_review_qty)
#     stars_list.append(star_review_stars)
#     stars_qty_list.append(star_review_qty)
#     visitor_review_qty_list.append(visitor_review_qty)

#     # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
#     except Exception as e:
#         print(f"{i}행 문제가 발생: {e}")
        
#         # 내역이 없으므로 null을 임시로 넣어줍니다.
#         naver_map_type_list.append('null')
#         blog_review_qty_list.append('null')
#         stars_list.append('null')
#         stars_qty_list.append('null')
#         visitor_review_qty_list.append('null')
        
driver.quit()
sub_driver.quit()

In [ ]:
print(dict_review)
print(len(dict_review['빵이 맛있어요']))

In [ ]:
print(naver_map_type_list)
print(len(naver_map_type_list))

In [ ]:
print(visitor_review_qty_list)
print(len(visitor_review_qty_list))

In [ ]:
df2['naver_map_type'] = naver_map_type_list            # 네이버 지도의 유형 리스트
df2['blog_review_qty_list'] = blog_review_qty_list     # 블로그 리뷰 수 리스트
df2['stars'] = stars_list                              # 별점 점수(2022년 종료 예정) 정보 리스트
df2['stars_qty'] = stars_qty_list                      # 별점 평가 수(2022년 종료 예정) 리스트
df2['visitor_review_qty'] = visitor_review_qty_list    # 방문자 평가 참여자수 리스트 


for key in dict_review:
    df2[key] = dict_review[key]

In [ ]:
print(stars_qty_list)

In [ ]:
df['naver_map_url'] = ''
df

In [ ]:
df.iloc[0,-1]